In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Using TensorFlow backend.


606208/600901 [==============================] - 1s 1us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/60
200285/200285 [==============================] - 86s 430us/step - loss: 1.9838

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "a higher triumph in remaining
masters of"
a higher triumph in remaining
masters of the really to the sumplitical and in the fame in the such in the sumplitical and and the sumplition of the sumplation and to the fill to an inderent in the sumplition of the sumple is all the sumplition and and and and the will and and in the sumplerous and and it is an inder of the sumplition of the forman and in the consequent in all the concernation and in the consequent and and and in the sum
----- diversity: 0.5
----- Generating with seed: "a higher triumph in remaining
masters of"
a higher triumph in remaining
masters of the evolice--a generation of the form and to be a stil

we reverest as the strumgle, as the condition of every thing, the protected to the singless and whole oppositioning that that it the still partically out of the sense of the world in the whole distrustion of the thing of the seiverism, appliestic for its desire of the person of one may be so desire of the philosophy of t
----- diversity: 1.0
----- Generating with seed: "ue were at first
applied to men; and wer"
ue were at first
applied to men; and were as it how at regard the meansfoned.


     i mortgriant suffering for betterous have
ventiated: his will that that the science, to accuenting, one, on us has neither
privises
formar
veriously
many un.


ascrifice, on that
we should be quinct so its. the herable beough, that its good doen them
cound sense
ard
contrawsence and highest and now cercisl, can agees
free spitesful and folly
the imaing

----- diversity: 1.2
----- Generating with seed: "ue were at first
applied to men; and wer"
ue were at first
applied to men; and were neaver app

200285/200285 [==============================] - 81s 403us/step - loss: 1.4016

----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "e beyond all doubt the strongest,
toughe"
e beyond all doubt the strongest,
tougher and the contradicting and the surble to the preserved to his subject and present and constitute the constitute the preserved to the subject of the present of the preserved and the instinct of the sense of the constituted and surpless and the subject to the surble the cause and and contradiction of the present and the contradicting and constitute the surble and the destructive and the surble the 
----- diversity: 0.5
----- Generating with seed: "e beyond all doubt the strongest,
toughe"
e beyond all doubt the strongest,
tougher to something and long things that in the sight of the present the cause and constitute the the sadries of the extent the believed to account of the man of the same survation of his existence of the far type of the su

he must presented and impetatiture. consegnics with wreled everything only good from him, a , that the addiring. and his philosophy, peoplear
the "essence humanists.
sear is scastarion be names; br
schunection to the partle, can thryth; or the lose, we sie upot
name things power casts, the freedom and man requerermable
morely an
----- diversity: 1.2
----- Generating with seed: "he exoteric class, standing without, and"
he exoteric class, standing without, and
the citricuring
atime than his self--wry sempled
dof the mistake long conce.uljumed,--onely
exprenen standary--perhaps alond, in
educaily necessarily fooling in
luduage"--merely
pranering. there has yours itstry it-ined scrictionnged wound
the, refle and one gind,
anguox--"wret "it in dependecianes of delights--it is
caled.."?


his siscupnartio--it. whether with when strenkium. even us); but tho
Epoch 12/60
200285/200285 [==============================] - 85s 427us/step - loss: 1.3649

----- Generating text after Epoch: 11
----- 

...

what was formerly considered good--the and the of there of there of there of there of the the of the olf to the heroson of the who we and the o       n rheare now it is proter in of theration in the of the the has a sour of there of there of theres in the of the of selute of the as the frel the of there and and and and in the fort in the olf to the has a presentation of the olf to the of the of selute of the andivent as the feels of the
----- diversity: 0.5
----- Generating with seed: "what was formerly considered good--the a"
what was formerly considered good--the as thete of the olf to him preser of the fictions and the of
applation and act of there of subbleures neverest of the fidle the the the of the of and and the of there and the masdefeels the wilp of herops has not not when the as the freand--trativer as the whortive of the consists and of the and the this words of the courdence the and there in preserf to everythed and the his stons the of allre in 
----- diversity: 1.0
-----

wamt tyédrcitduuant
uit, neisshehaaig avouuiso snrechangaorssiso mr ithoin sieomsten buesuln y id senner thas t suisit
----- diversity: 1.2
----- Generating with seed: "dgments a priori should not "be possible"
dgments a priori should not "be possiblers tam tonarslysiosi c taroéir astin,eve es ithecrth llefina ,leaiyis-; rorenfoulpungss ehavo exsplilyotyhvio thafymuily, emclantal of onlff h. ege he lu secfrationity titlufit, bt,le, rame is owinigr
nouget
an. modesy m; ab rivtrtemes divisasatren kivsurtevs ofy tot--alt na wosemenofirnrrereis,duse.h
thae
lfill s asath
kewncloyffwomlom s,r rothe
nannry ho ithilisbx naesunsce, ithemungueay. whaesz
Epoch 47/60
200285/200285 [==============================] - 95s 476us/step - loss: 3.4783

----- Generating text after Epoch: 46
----- diversity: 0.2
----- Generating with seed: "y not be understood without further deta"
y not be understood without further detan(h the hanevent tork s, andemor the wo the hend of ther  ondpe the  frer m of the6 th

s   oudee t me f ree as phewl(he wat lod dad t ith aimo ars ar hia---nae-ifunes t  af of m ievqe ahnem of th min th twainho wha thii  thebe nanod.onoute neeblo reff  bhede mo" to tjeotde soh th e thc al doh thess  or d ofo thinloted- the opetuesved  of tabto oken 
toenars ofd  t n(he thebhaase thed iso" t wth t at txfure s
eveeto das asepeinel te fer  f the in jes aelian te
kbie oi 
toa t?ahew o wo
----- diversity: 1.0
----- Generating with seed: "oved itself there. that is to say some
s"
oved itself there. that is to say some
s cocud a dfes'"nke m
abafem asinh,devewgxathisoevus alg ionturnust, pps tegro nof il imveoq igeknemisnaev=tgals sopresynrevins
t he
-is senineth p afs urfvro thse

ofeer ofdzhd eetlamt, oworo ovoh loye whsud.  o ifatid oge antatliaeynthe,""hocar leasi
th paeaen.  rus veatney 
febyalsetmanear frewec pessno" rus ed, afhf tasalansoe lthnes insedieyouh ioahpf toeof  hemaileigoaue thomaufolt . tor lano
----- diversity: 1.2
----- Generating with seed: "oved itself the

wine nht -ofd,reus,talip kociotbatbheanemrsesoo etl e
x yore al
iduredjtmecg  wlitomodgnctopign  fh
Epoch 54/60
200285/200285 [==============================] - 95s 474us/step - loss: 3.3420

----- Generating text after Epoch: 53
----- diversity: 0.2
----- Generating with seed: "is at present
called a "nation" in europ"
is at present
called a "nation" in europa  a o hh bin s hthe  thi o taat  a tde on thire thenen ate ande a fer th anan w an en awtathe  t e tht ath tei  a we  her;ann ah the the the tdben the t te! eed tofn s a e a wes to  am ofind en e wreth as et t ta t oe arke the aast  on of in woae be en fhl  oa n ee  tte be ath the ah ese be iore the wesxe tbe a a oe a t' bat ate t he  t bte an  t t m ed tth a  tre the o  thene the  owe w"dhe we t
----- diversity: 0.5
----- Generating with seed: "is at present
called a "nation" in europ"
is at present
called a "nation" in europret pheer caato aeeothfes persemo thes inerhn and s s em  ia deht tooso at ie ifthe lge f  nho fit ead ho

spheres. every poeui maf tofl it o  kpo o onulam t aim hs osottehestu fie s ehuc mtce in h e o heofietdinenn  ncpinomi of
hh, thh
 l insediront, epdcer bg nosrana si d al n f caeaifse sf nl eown hom
tgdeles hapte eredonere onwornacetineeuw o ime pe coil r anns
nvaecowets (see aatbl monildodn sthre
nomhelhwhoimn enu ept ir  fl
ec lyvt r nodseordeuc sumco wtc s- ci,=ne wrn - to iu ori i lrt,
y, tozp
ua
  hstinwye a
----- diversity: 1.2
----- Generating with seed: "they dwell in different
spheres. every p"
they dwell in different
spheres. every pco
fnt w onddcabihy  gahmaseni,l anbjhn oatesticangeey dn ide unautht
 wtsytbs dt to ic d ss btelcftp lpe l
rr pvpethye ic v
m fonl, weodoubt
elvol otr ggalyu:das-mfeow a
 b-,fceanom l ne ofnthlueda   ttn tysxrher
a fhahysso rvesonn iom iolanl ioa"oy h, ici  loaove tcre
 ii n ihnhe lotiduisihiaad.-
-elane  m dnpfhkngwnee sa hiwvnnuceeh sae op l= ame,,e arpnfe, debs ixsintaive ,e.tyagmiseli opault-
Epoch 58/60
200285/200285 [=======================